In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Alzheimers_Disease"
cohort = "GSE109887"

# Input paths
in_trait_dir = "../../input/GEO/Alzheimers_Disease"
in_cohort_dir = "../../input/GEO/Alzheimers_Disease/GSE109887"

# Output paths
out_data_file = "../../output/preprocess/Alzheimers_Disease/GSE109887.csv"
out_gene_data_file = "../../output/preprocess/Alzheimers_Disease/gene_data/GSE109887.csv"
out_clinical_data_file = "../../output/preprocess/Alzheimers_Disease/clinical_data/GSE109887.csv"
json_path = "../../output/preprocess/Alzheimers_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Determine if gene expression data is available
# Based on the background information, this dataset contains gene expression data from Illumina HumanHT-12 V4.0
is_gene_available = True

# 2. Data Availability and Type Conversion Functions
# 2.1 Identify rows in sample characteristics where data is recorded
trait_row = 3  # The trait (AD vs Control) is in row 3 as 'disease state'
age_row = 1    # Age is in row 1
gender_row = 0  # Gender is in row 0

# 2.2 Data type conversion functions
def convert_trait(value):
    """Convert trait values to binary (0 for Control, 1 for AD)"""
    if not isinstance(value, str):
        return None
    
    # Split by colon and get the value part
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary
    if value.lower() == "ad" or value.lower() == "alzheimer's disease":
        return 1
    elif value.lower() == "control":
        return 0
    else:
        return None

def convert_age(value):
    """Convert age values to continuous numeric values"""
    if not isinstance(value, str):
        return None
    
    # Split by colon and get the value part
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to float if possible
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender values to binary (0 for Female, 1 for Male)"""
    if not isinstance(value, str):
        return None
    
    # Split by colon and get the value part
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary
    if value.lower() == "male":
        return 1
    elif value.lower() == "female":
        return 0
    else:
        return None

# 3. Save metadata
# Determine if trait data is available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None:
    # Define the sample characteristics dictionary from the previous output
    sample_char_dict = {
        0: ['gender: Male', 'gender: Female'], 
        1: ['age: 91', 'age: 87', 'age: 82', 'age: 73', 'age: 94', 'age: 72', 'age: 90', 'age: 86', 
            'age: 92', 'age: 81', 'age: 95', 'age: 75', 'age: 77', 'age: 84', 'age: 85', 'age: 89', 
            'age: 78', 'age: 70', 'age: 88', 'age: 79'], 
        2: ['tissue: brain, middle temporal gyrus'], 
        3: ['disease state: AD', 'disease state: Control']
    }
    
    # Create a compatible DataFrame for geo_select_clinical_features
    # The function expects a DataFrame where rows are features and columns are samples
    # For this test case, we'll create a minimal DataFrame with the expected structure
    # Create a dummy DataFrame with the right structure
    data = {}
    for i in range(2):  # Create 2 sample columns for testing
        col_name = f"GSM{i+1}"
        data[col_name] = [
            sample_char_dict[0][i % len(sample_char_dict[0])],  # Gender
            sample_char_dict[1][i % len(sample_char_dict[1])],  # Age
            sample_char_dict[2][0],                             # Tissue (constant)
            sample_char_dict[3][i % len(sample_char_dict[3])]   # Disease state
        ]
    
    clinical_data = pd.DataFrame(data)
    
    # Extract clinical features using the geo_select_clinical_features function
    selected_clinical_data = geo_select_clinical_features(
        clinical_data, 
        trait=trait, 
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    print("Preview of extracted clinical data:")
    print(preview_df(selected_clinical_data))
    
    # Save the clinical data to CSV
    selected_clinical_data.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
# Review the gene identifiers
# Looking at the first 20 identifiers shows a mix of conventional gene symbols like A1BG, A2M, AAAS, AACS
# These appear to be standard human gene symbols (HGNC symbols)
# Names like A1CF, A2BP1, etc. are older gene symbols that have been updated in newer nomenclature
# There are also some less common identifiers like 7A5 and AAA1, but overall these appear to be gene symbols

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [ ]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Since we determined that gene mapping is not required (requires_gene_mapping = False),
# we can directly normalize the gene symbols in the index
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Original gene data shape: {gene_data.shape}")
print(f"Normalized gene data shape: {normalized_gene_data.shape}")

# Save the processed gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Processed gene data saved to {out_gene_data_file}")

# 4. Re-extract clinical data properly
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
_, clinical_raw = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Define conversion functions again to ensure they're available
def convert_trait(value):
    """Convert trait values to binary (0 for Control, 1 for AD)"""
    if not isinstance(value, str):
        return None
    
    # Split by colon and get the value part
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary
    if value.lower() == "ad" or value.lower() == "alzheimer's disease":
        return 1
    elif value.lower() == "control":
        return 0
    else:
        return None

def convert_age(value):
    """Convert age values to continuous numeric values"""
    if not isinstance(value, str):
        return None
    
    # Split by colon and get the value part
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to float if possible
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender values to binary (0 for Female, 1 for Male)"""
    if not isinstance(value, str):
        return None
    
    # Split by colon and get the value part
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary
    if value.lower() == "male":
        return 1
    elif value.lower() == "female":
        return 0
    else:
        return None

# Extract clinical features properly
clinical_data = geo_select_clinical_features(
    clinical_raw, 
    trait=trait, 
    trait_row=3,  # From previous step
    convert_trait=convert_trait,
    age_row=1,    # From previous step
    convert_age=convert_age,
    gender_row=0, # From previous step
    convert_gender=convert_gender
)

print(f"Clinical data shape: {clinical_data.shape}")
print("Clinical data preview:")
print(preview_df(clinical_data.T))

# Save the extracted clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_data.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 5. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_data, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 6. Handle missing values in the linked data
# The trait column name should be 'Alzheimers_Disease' based on the variable we passed to geo_select_clinical_features
linked_data = handle_missing_values(linked_data, trait_col=trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 7. Evaluate whether the trait and demographic features are biased
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 8. Conduct final quality validation and save cohort info
note = "Gene expression data from Illumina HumanHT-12 V4.0 in middle temporal gyrus brain tissue."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 9. Save the linked data if it is usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file, index=True)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset {cohort} was determined to be unusable due to bias or other issues. Data not saved.")